# Table of Contents
### pygame Initialization and set up display
### Define a font to render text 
### Colors
### Load images
### Load furniture images
### Default positions for furniture items (using -1 to indicate no placement yet)
### Define initial positions list for all desks and sofas
### Adjust robot size and scale
### Room grid, dust, robots, and obstacles
### Function to randomly place dust particles on the grid
### Class Robot
### Robot Moving
### Collision Function
### Moving Obstacle Class
### Class ActorAgent
### Room setup for grid, walls, and openings
### Main Loop


 # pygame Initialization and set up display


In [ ]:

SCREEN_WIDTH, SCREEN_HEIGHT = 750, 750
ROWS, COLS = 40, 40  # Grid size
TILE_SIZE = SCREEN_WIDTH // COLS # Size of each grid tile
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Cleaning Robot Simulation ")

 # Define a font to render text 


In [29]:
font = pygame.font.SysFont("Arial", 12)




# Colors


In [ ]:
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (128, 128, 128)
BROWN = (165, 42, 42)
BEIGE = (165, 142, 102) 

# Load images


In [ ]:
ROBOT_IMAGE = pygame.image.load('Downloads/robotC.png')  # Path to robot image
OBSTACLE_IMAGE = pygame.image.load('Downloads/obstacleC.png')  # Path toobstacle image
DUST_IMAGE = pygame.image.load('Downloads/dustC.png')  # Path to  dust image
DUST_IMAGE = pygame.transform.scale(DUST_IMAGE, (TILE_SIZE, TILE_SIZE))


# Load furniture images


In [ ]:
DESK_IMAGE = pygame.image.load('Downloads/dinnerC.png')
DESK_IMAGE = pygame.transform.scale(DESK_IMAGE, (TILE_SIZE * 7, TILE_SIZE * 6))
DESK1_IMAGE = pygame.image.load('Downloads/meetingC.png')
DESK1_IMAGE = pygame.transform.scale(DESK1_IMAGE, (TILE_SIZE * 8, TILE_SIZE * 8))
DESK2_IMAGE = pygame.image.load('Downloads/deskC.png')
DESK2_IMAGE = pygame.transform.scale(DESK2_IMAGE, (TILE_SIZE * 7, TILE_SIZE * 7))
DESK3_IMAGE = pygame.image.load('Downloads/deskC1.png')
DESK3_IMAGE = pygame.transform.scale(DESK3_IMAGE, (TILE_SIZE * 4, TILE_SIZE * 4))
SOFA_IMAGE = pygame.image.load('Downloads/sofaC.png')
SOFA_IMAGE = pygame.transform.scale(SOFA_IMAGE, (TILE_SIZE * 8, TILE_SIZE * 8))
DESK4_IMAGE = pygame.image.load('Downloads/deskC2.png')
DESK4_IMAGE = pygame.transform.scale(DESK4_IMAGE, (TILE_SIZE * 4, TILE_SIZE * 4))
DESK5_IMAGE = pygame.image.load('Downloads/deskC3.png')
DESK5_IMAGE = pygame.transform.scale(DESK5_IMAGE, (TILE_SIZE * 4, TILE_SIZE * 5))
DESK6_IMAGE = pygame.image.load('Downloads/deskC4.png')
DESK6_IMAGE = pygame.transform.scale(DESK6_IMAGE, (TILE_SIZE * 6, TILE_SIZE * 7))
FURN_IMAGE = pygame.image.load('Downloads/furnC.png')
FURN_IMAGE = pygame.transform.scale(FURN_IMAGE, (TILE_SIZE * 7, TILE_SIZE * 2))
FURN1_IMAGE = pygame.image.load('Downloads/furnC1.png')
FURN1_IMAGE = pygame.transform.scale(FURN1_IMAGE, (TILE_SIZE * 8, TILE_SIZE * 2))
FURN2_IMAGE = pygame.image.load('Downloads/furnC2.png')
FURN2_IMAGE = pygame.transform.scale(FURN2_IMAGE, (TILE_SIZE * 8, TILE_SIZE * 2))
FURN3_IMAGE = pygame.image.load('Downloads/furnC3.png')
FURN3_IMAGE = pygame.transform.scale(FURN3_IMAGE, (TILE_SIZE * 2, TILE_SIZE * 8))


# Default positions for furniture items (using -1 to indicate no placement yet)


In [ ]:
desk_x = desk_y = desk1_x = desk1_y = desk2_x = desk2_y = desk3_x = desk3_y = -1
sofa_x = sofa_y = desk4_x = desk4_y = desk5_x = desk5_y = desk6_x = desk6_y = furn_x = furn_y = furn1_x = furn1_y = furn2_x = furn2_y = furn2_x = furn2_y = -1


# Define initial positions list for all desks and sofas


In [ ]:
desk_positions = [(0, 0)] * 6 
sofa_position = (-1, -1) 



# Adjust robot size and scale


In [ ]:
ROBOT_IMAGE_SIZE = TILE_SIZE  # Change size  to make the robot bigger
ROBOT_IMAGE = pygame.transform.scale(ROBOT_IMAGE, (ROBOT_IMAGE_SIZE, ROBOT_IMAGE_SIZE))

OBSTACLE_IMAGE = pygame.transform.scale(OBSTACLE_IMAGE, (TILE_SIZE, TILE_SIZE))


# Room grid, dust, robots, and obstacles


In [ ]:
grid = [[0 for _ in range(COLS)] for _ in range(ROWS)]
dust_positions = []
robots = []
moving_obstacles = []


# Function to randomly place dust particles on the grid


In [ ]:
# to store the positions of all robots
robot_positions = {}

def spawn_dust():
    for _ in range(150):  # Number of dust particles
        x, y = random.randint(0, ROWS - 1), random.randint(0, COLS - 1)
        if grid[x][y] == 0:  # Ensure it's an empty spot
            dust_positions.append((x, y))


# Class Robot

In [ ]:
class Robot:
    def __init__(self, x, y, id):
        self.x, self.y = x, y
        self.id = id
        self.agent = robot_agent
        self.cleaned_tiles = {}  # To track cleaned tiles
        self.total_reward = 0  # Track total reward for the robot
        robot_positions[self.id] = (self.x, self.y)

    def move_towards(self, target):
        # Occasionally take a random action to encourage exploration
        if random.random() < 0.1:  # 10% of the time take random action
            action = random.choice(["up", "down", "left", "right"])
        else:
            state = (self.x, self.y)
            action = self.agent.choose_action(state)  # Follow the chosen action from agent

        self.take_action(action)

    def take_action(self, action):
        global dust_positions
        new_x, new_y = self.x, self.y
        reward = 0
        if action == "up":
            new_x -= 1
        elif action == "down":
            new_x += 1
        elif action == "left":
            new_y -= 1
        elif action == "right":
            new_y += 1

        # Validate move
        if 0 <= new_x < ROWS and 0 <= new_y < COLS and grid[new_x][new_y] != 2:
            if not will_collide((new_x, new_y)) and not is_occupied_by_robot((new_x, new_y), self.id):
                # Positive reward for valid moves
                reward = 1
                self.x, self.y = new_x, new_y
                robot_positions[self.id] = (self.x, self.y)

                # Bonus reward for cleaning dust
                if (self.x, self.y) in dust_positions:
                    reward += 5
                    dust_positions.remove((self.x, self.y))

                    # Update cleaned tiles and dust positions
                    self.cleaned_tiles[(self.x, self.y)] = self.cleaned_tiles.get((self.x, self.y), 0) + 1
                    dust_positions = [d for d in dust_positions if d != (self.x, self.y)]

        # Negative reward for invalid moves
        else:
            reward = -2

        # Update the agent with state, action, reward, and next state
        next_state = (self.x, self.y)
        done = len(dust_positions) == 0  # Check if the task is complete
        self.agent.update((self.x, self.y), action, next_state, reward, done)
        # Update the total reward
        self.total_reward += reward


    def clean_dust(self):
        # Remove dust if robot is on a dust position
        global dust_positions
        dust_positions = [d for d in dust_positions if d != (self.x, self.y)]

    def draw(self):
        # Draw the robot's image at its current position
        screen.blit(ROBOT_IMAGE, (self.y * TILE_SIZE, self.x * TILE_SIZE))
        # Display the total accumulated reward near the robot
        reward_text = font.render(f"Reward: {self.total_reward}", True, (255, 0, 0))
        screen.blit(reward_text, (self.y * TILE_SIZE, self.x * TILE_SIZE - 20))

    def update_path(self):
        """
        Update the robot's path by checking for obstacles and avoiding them.
        """
        if self.path and (self.x, self.y) != self.path[0]:
            target = self.path[0]
            
            # Check if the target is an obstacle or blocked
            if self.is_obstacle_in_path(target):
                # Update the path by skipping the blocked cell (simple strategy)
                self.path = self.get_alternate_path()

        # Default case: continue on the current path if it's valid
        if self.path:
            next_position = self.path[0]
            if not self.will_collide(next_position):
                self.x, self.y = next_position
                robot_positions[self.id] = (self.x, self.y)

                # Remove the dust from the current position if present
                self.dust_positions = [d for d in self.dust_positions if d != (self.x, self.y)]

    def get_alternate_path(self):
        """
        Simple method to find an alternate path when an obstacle is detected. 
        In this version, we just move towards the closest valid point (ignoring the previous path).
        """
        valid_moves = self.get_neighbors((self.x, self.y))  # Get the valid neighbors around the robot
        if valid_moves:
            # Pick a valid move and return a simple "path" containing that move
            return [valid_moves[0]]  # Moving to the first valid neighboring cell
        return []

    def choose_target(self):
        # Prioritize dust that hasn't been cleaned often
        if dust_positions:
            target = min(dust_positions, key=lambda dust: self.cleaned_tiles.get(dust, 0))  # Less cleaned target
            return target
        return None


# Robot Moving

In [ ]:
def move_robots():
    for robot in robots:
        # Get the target with the least cleaning history
        target = robot.choose_target()
        if target:
            robot.move_towards(target)



# Collision Function

In [ ]:
def will_collide(next_position):
    """
    Check if any moving obstacle is currently or will be at the next_position.
    """
    for obstacle in moving_obstacles:
        # Future position of the moving obstacle based on its direction
        future_x, future_y = obstacle.x + obstacle.direction[0], obstacle.y + obstacle.direction[1]
        
        # Check for collision with both current and future positions of the obstacle
        if (obstacle.x, obstacle.y) == next_position or (future_x, future_y) == next_position:
            return True
    return False

   # function to check if a position is occupied by another robot


In [ ]:
def is_occupied_by_robot(position, robot_id):
    for other_robot_id, other_position in robot_positions.items():
        if other_robot_id != robot_id and other_position == position:
            return True
    return False

# Moving Obstacle Class

In [ ]:
class MovingObstacle:
    def __init__(self, x, y):
        self.x, self.y = x, y
        self.direction = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])

    def is_near_robot(self, robots):
        for robot in robots:
            if abs(robot.x - self.x) <= 1 and abs(robot.y - self.y) <= 1:
                return True
        return False

    def move(self, robots):
        if random.random() < 0.1:
            self.direction = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])
        
        # Calculate new position
        new_x, new_y = self.x + self.direction[0], self.y + self.direction[1]
        
        # Check if the new position is within bounds, not a wall, and away from robots
        if (0 <= new_x < ROWS and 0 <= new_y < COLS and 
            grid[new_x][new_y] != 2 and not self.is_near_robot(robots)):
            self.x, self.y = new_x, new_y
        else:
            # Choose a new direction if the current one would lead to a collision
            self.direction = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])

    def draw(self):
        screen.blit(OBSTACLE_IMAGE, (self.y * TILE_SIZE, self.x * TILE_SIZE))


# Class ActorAgent

In [ ]:
class ActorCriticAgent:
    def __init__(self, alpha=0.01, beta=0.01, gamma=0.99):
        self.alpha = alpha  # Learning rate for the Actor
        self.beta = beta    # Learning rate for the Critic
        self.gamma = gamma  # Discount factor
        self.policy = {}    # Policy (Actor's parameters)
        self.value_function = {}  # Value function (Critic's parameters)

    def get_policy(self, state):
        # Softmax policy for action probabilities
        if state not in self.policy:
            self.policy[state] = {action: 0 for action in ["up", "down", "left", "right"]}
        return self.policy[state]

    def get_value(self, state):
        # Default value function initialization
        return self.value_function.get(state, 0)

    def choose_action(self, state):
        policy = self.get_policy(state)
        exp_values = np.exp(list(policy.values()))
        probabilities = exp_values / np.sum(exp_values)
        actions = list(policy.keys())
        return np.random.choice(actions, p=probabilities)


    def update(self, state, action, next_state, reward, done):
        policy = self.get_policy(state)
        value = self.get_value(state)
        next_value = self.get_value(next_state) if not done else 0

        # Compute Temporal Difference (TD) Error
        td_error = reward + self.gamma * next_value - value

        # Update Critic (Value Function)
        self.value_function[state] = value + self.beta * td_error

        # Update Actor (Policy)
        policy[action] += self.alpha * td_error

        

# Actor-Critic Agent object

In [ ]:

robot_agent = ActorCriticAgent()


# Validation Funtion

In [ ]:
def is_valid_point(point, dynamic_grid):
    return 0 <= point[0] < ROWS and 0 <= point[1] < COLS and dynamic_grid[int(point[0])][int(point[1])] == 0



In [ ]:
def get_neighbors(pos, dynamic_grid):
    neighbors = [(pos[0] + dx, pos[1] + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]
    return [(r, c) for r, c in neighbors if 0 <= r < ROWS and 0 <= c < COLS and dynamic_grid[r][c] != 2]


# Room setup for grid, walls, and openings


In [ ]:
def create_rooms():
    global desk_x, desk_y, desk1_x, desk1_y, desk2_x, desk2_y, desk3_x, desk3_y, sofa_x, sofa_y, desk4_x, desk4_y, desk5_x, desk5_y, desk6_x, desk6_y, furn_x, furn_y, furn1_x, furn1_y, furn2_x, furn2_y, furn3_x, furn3_y
    mid_col = COLS // 2
    mid_row = ROWS // 2

    # Create walls dividing the rooms
    for row in range(ROWS):
        if not (mid_row - 16 <= row <= mid_row + 2):
            grid[row][mid_col] = 2  # Add a wall in the middle column
            
    # door opening in the middle wall
    grid[ROWS - 5][mid_col] = 0  # Open door
    grid[ROWS - 4][mid_col] = 0  # Open door

    for col in range(COLS):
        if not (1 <= col <= 2 or (COLS // 2 - 1 <= col <= COLS // 2)):
            grid[mid_row][col] = 2  # Add a wall across the middle row

    small_room_width = COLS // 3
    small_room_height = ROWS // 3

    for row in range(small_room_height):
        if row == 1 or row == 2:
            continue
        grid[row][small_room_width] = 2  # Left small room
        grid[row][COLS - small_room_width - 1] = 2  # Right small room

    for col in range(small_room_width):
        if col == 1 or col == 2:
            continue
        grid[small_room_height][col] = 2  # Left small room
        grid[small_room_height][COLS - col - 1] = 2  # Right small room

    # additional walls and doors based on your configuration
    wall_rec1_x = mid_row - 13
    wall_rec1_y = mid_col -14
    wall_width1 = 7
    wall_height1 = 7

    if wall_rec1_x + wall_height1 <= ROWS and wall_rec1_y + wall_width1 <= COLS:
        for i in range(wall_height1):
            for j in range(wall_width1):
                grid[wall_rec1_x + i][wall_rec1_y + j] = 2

    wall_rec_x = mid_row + 7
    wall_rec_y = mid_col + 7
    wall_width = 8
    wall_height = 8

    if wall_rec_x + wall_height <= ROWS and wall_rec_y + wall_width <= COLS:
        for i in range(wall_height):
            for j in range(wall_width):
                grid[wall_rec_x + i][wall_rec_y + j] = 2

    wall_sq_x = mid_row - 14
    wall_sq_y = mid_col + 7

    for i in range(7):
        for j in range(7):
            grid[wall_sq_x + i][wall_sq_y + j] = 2

    wall_sq_x1 = mid_row - 20
    wall_sq_y1 = mid_col + 14

    for i in range(4):
        for j in range(4):
            grid[wall_sq_x1 + i][wall_sq_y1 + j] = 2

    wall_sofa_x1 = mid_row + 12
    wall_sofa_y1 = mid_col - 20

    for i in range(8):
        for j in range(8):
            grid[wall_sofa_x1 + i][wall_sofa_y1 + j] = 2

    wall_sq_X2 = mid_row + 1
    wall_sq_y2 = mid_col + 16
    sq_width = 4
    sq_height = 4

    if wall_sq_X2 + sq_height <= ROWS and wall_sq_y2 + sq_width <= COLS:
        for i in range(sq_height):
            for j in range(sq_width):
                grid[wall_sq_X2 + i][wall_sq_y2 + j] = 2

    wall_sq_x3 = mid_row - 14
    wall_sq_y3 = mid_col - 6

    for i in range(5):
        for j in range(4):
            grid[wall_sq_x3 + i][wall_sq_y3 + j] = 2

    
    wall_sq_x4 = mid_row + 8
    wall_sq_y4 = mid_col - 6

    for i in range(7):
        for j in range(6):
            grid[wall_sq_x4 + i][wall_sq_y4 + j] = 2

    
    wall_sq_X5 = mid_row + 1
    wall_sq_y5 = mid_col - 13
    sq_width5 = 7
    sq_height5 = 2

    if wall_sq_X5 + sq_height5 <= ROWS and wall_sq_y5 + sq_width5 <= COLS:
        for i in range(sq_height5):
            for j in range(sq_width5):
                grid[wall_sq_X5 + i][wall_sq_y5 + j] = 2

    wall_sq_X6 = mid_row + 18
    wall_sq_y6 = mid_col + 3
    sq_width6 = 8
    sq_height6 = 2

    if wall_sq_X6 + sq_height6 <= ROWS and wall_sq_y6 + sq_width6 <= COLS:
        for i in range(sq_height6):
            for j in range(sq_width6):
                grid[wall_sq_X6 + i][wall_sq_y6 + j] = 2

    wall_sq_X7 = mid_row - 20
    wall_sq_y7 = mid_col -  18
    sq_width7 = 8
    sq_height7 = 2

    if wall_sq_X7 + sq_height7 <= ROWS and wall_sq_y7 + sq_width7 <= COLS:
        for i in range(sq_height7):
            for j in range(sq_width7):
                grid[wall_sq_X7 + i][wall_sq_y7 + j] = 2

    wall_sq_X8 = mid_row - 20
    wall_sq_y8 = mid_col - 20
    sq_width8 = 2
    sq_height8 = 8

    if wall_sq_X8 + sq_height8 <= ROWS and wall_sq_y8 + sq_width8 <= COLS:
        for i in range(sq_height8):
            for j in range(sq_width8):
                grid[wall_sq_X8 + i][wall_sq_y8 + j] = 2




     # Place the desk image above the wall
    desk_x = wall_rec1_x  # Align desk with the wall (same x position)
    desk_y = wall_rec1_y  # Align desk with the wall (same y position)
    desk1_x = wall_rec_x  # Align desk with the wall (same x position)
    desk1_y = wall_rec_y  # Align desk with the wall (same y position)
    desk2_x = wall_sq_x  # Align desk with the wall (same x position)
    desk2_y = wall_sq_y  # Align desk with the wall (same y position)
    desk3_x = wall_sq_x1  # Align desk with the wall (same x position)
    desk3_y = wall_sq_y1 # Align desk with the wall (same y position)
    sofa_x = wall_sofa_x1
    sofa_y = wall_sofa_y1
    desk4_x = wall_sq_X2  # Align desk with the wall (same x position)
    desk4_y = wall_sq_y2 # Align desk with the wall (same y position)
    desk5_x = wall_sq_x3  # Align desk with the wall (same x position)
    desk5_y = wall_sq_y3 # Align desk with the wall (same y position)
    desk6_x = wall_sq_x4  # Align desk with the wall (same x position)
    desk6_y = wall_sq_y4 # Align desk with the wall (same y position)
    furn_x = wall_sq_X5
    furn_y = wall_sq_y5
    furn1_x = wall_sq_X6
    furn1_y = wall_sq_y6
    furn2_x = wall_sq_X7
    furn2_y = wall_sq_y7
    furn3_x = wall_sq_X8
    furn3_y = wall_sq_y8


# Main Loop


In [ ]:
def main():
    global dust_positions, robots
    create_rooms()
    spawn_dust()

    # Create robots at different starting positions
    robots.append(Robot(5, 5, 1))
    robots.append(Robot(ROWS - 2, COLS - 3, 2))
    robots.append(Robot(ROWS // 2, COLS // 2 - 10, 3))
   # Create moving obstacles without grid marking
    for _ in range(5):
        moving_obstacles.append(MovingObstacle(random.randint(0, ROWS-1), random.randint(0, COLS-1)))

    clock = pygame.time.Clock()
    running = True

    while running:
        screen.fill(WHITE) # Fill screen background
        draw_grid()

        # Move the robots towards the dust
        move_robots()
  
        # Update and draw robots
        for robot in robots:
            robot.draw() 

        # Draw and move obstacles
        for obstacle in moving_obstacles:
            obstacle.move(robots)
            obstacle.draw()

        # Draw dust as images
        for (x, y) in dust_positions:
            screen.blit(DUST_IMAGE, (y * TILE_SIZE, x * TILE_SIZE))

        

        # Draw desks and sofas
        screen.blit(DESK_IMAGE, (desk_y * TILE_SIZE, desk_x * TILE_SIZE))
        screen.blit(DESK1_IMAGE, (desk1_y * TILE_SIZE, desk1_x * TILE_SIZE))
        screen.blit(DESK2_IMAGE, (desk2_y * TILE_SIZE, desk2_x * TILE_SIZE))
        screen.blit(DESK3_IMAGE, (desk3_y * TILE_SIZE, desk3_x * TILE_SIZE))
        screen.blit(SOFA_IMAGE, (sofa_y * TILE_SIZE, sofa_x * TILE_SIZE))
        screen.blit(DESK4_IMAGE, (desk4_y * TILE_SIZE, desk4_x * TILE_SIZE))
        screen.blit(DESK5_IMAGE, (desk5_y * TILE_SIZE, desk5_x * TILE_SIZE))
        screen.blit(DESK6_IMAGE, (desk6_y * TILE_SIZE, desk6_x * TILE_SIZE))
        screen.blit(FURN_IMAGE, (furn_y * TILE_SIZE, furn_x * TILE_SIZE))
        screen.blit(FURN1_IMAGE, (furn1_y * TILE_SIZE, furn1_x * TILE_SIZE))
        screen.blit(FURN2_IMAGE, (furn2_y * TILE_SIZE, furn2_x * TILE_SIZE))
        screen.blit(FURN3_IMAGE, (furn3_y * TILE_SIZE, furn3_x * TILE_SIZE))
        pygame.display.flip()
        clock.tick(10)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

    pygame.display.flip()  # Update the display
    clock.tick(60)  # Cap the frame rate

    pygame.quit()

def draw_grid():
    for row in range(ROWS):
        for col in range(COLS):
            color = BEIGE if grid[row][col] == 2 else WHITE  # Use white for all grid squares
            pygame.draw.rect(screen, color, (col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE))

        
if __name__ == "__main__":
    main()